# Hysteresis Processing

This notebook processes raw hysteresis loop data following the protocols of [Jackson and Solheid, 2010](https://doi.org/10.1029%2F2009gc002932). Summary hysteresis parameters are calculated from a collection of loops. It illustrates the `rmag.process_hyst_loop` function that applies the steps detailed in the [hysteresis_processing_walk_through.ipynb](./hysteresis_processing_walk_through.ipynb) notebook.

## Install and import packages

In [1]:
import pmagpy.rockmag as rmag
import pmagpy.ipmag as ipmag
import pmagpy.contribution_builder as cb
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook(hide_banner=True)

## Import local data in MagIC format
In this demonstration we will be using data is from the following publication:
- Swanson-Hysell, N. L., Avery, M. S., Zhang, Y., Hodgin, E. B., Sherwood, R. J., Apen, F. E., et al. (2021). The paleogeography of Laurentia in its early years: New constraints from the Paleoproterozoic East-Central Minnesota Batholith. Tectonics, 40, e2021TC006751. https://doi.org/10.1029/2021TC006751

We will use a downloaded MagIC contribution that is within the directory specified below.

In [2]:
# set the dir_path to the directory where the measurements.txt file is located
dir_path = '../example_data/ECMB'

# set the name of the MagIC file
ipmag.unpack_magic('magic_contribution_20213.txt', 
                     dir_path = dir_path,
                     input_dir_path = dir_path,
                     print_progress=False)

# create a contribution object from the tables in the directory
contribution = cb.Contribution(dir_path)
measurements = contribution.tables['measurements'].df
specimens = contribution.tables['specimens'].df

1  records written to file  /Users/penokean/0000_GitHub/RockmagPy-notebooks/example_data/ECMB/contribution.txt
1  records written to file  /Users/penokean/0000_GitHub/RockmagPy-notebooks/example_data/ECMB/locations.txt
90  records written to file  /Users/penokean/0000_GitHub/RockmagPy-notebooks/example_data/ECMB/sites.txt
312  records written to file  /Users/penokean/0000_GitHub/RockmagPy-notebooks/example_data/ECMB/samples.txt
1574  records written to file  /Users/penokean/0000_GitHub/RockmagPy-notebooks/example_data/ECMB/specimens.txt
17428  records written to file  /Users/penokean/0000_GitHub/RockmagPy-notebooks/example_data/ECMB/measurements.txt
-I- Using online data model
-I- Getting method codes from earthref.org
-I- Importing controlled vocabularies from https://earthref.org


## Get hysteresis data using the method code

The method codes relevant to hysteresis loops are:
 - `LP-HYS` for regular hysteresis loops
 - `LP-HYS-O` for hysteresis loops as a function of orientation
 - `LP-HYS-T` for hysteresis loops as a function of temperature

We can filter the data to only have data for `LP-HYS`.

In [3]:
hyst_measurements = measurements[measurements['method_codes'] == 'LP-HYS']

We can see what experimental hysteresis data are available in this dataset:

In [4]:
hyst_experiments = rmag.make_experiment_df(hyst_measurements)
hyst_experiments

,specimen,method_codes,experiment
0,NED1-5c,LP-HYS,IRM-VSM3-LP-HYS-218845
1,NED18-2c,LP-HYS,IRM-VSM3-LP-HYS-218847
2,NED2-8c,LP-HYS,IRM-VSM3-LP-HYS-218849
3,NED4-1c,LP-HYS,IRM-VSM3-LP-HYS-218853
4,NED6-6c,LP-HYS,IRM-VSM3-LP-HYS-218855


## Process hysteresis data for one experiment

We can visualize that data from one of the experiments using the `rmag.process_hyst_loop` function. The details behind the function are detailed in the hysteresis_processing_walk_through.ipynb notebook. The resulting `NED1_5c_hyst_process_result` is a dictionary that contains the processed data and calculated parameters.

In [5]:
#specify the experiment name and specimen name
experiment_name = 'IRM-VSM3-LP-HYS-218845'
specimen_name = 'NED1-5c'

experiment_hyst = measurements[measurements['experiment'] == experiment_name].reset_index(drop=True)
experiment_results = rmag.process_hyst_loop(experiment_hyst['meas_field_dc'].values, experiment_hyst['magn_mass'].values, specimen_name)
NED1_5c_hyst_process_result = experiment_results

## Process hysteresis data for all experiments

By providing the experiments dataframe and the measurements dataframe we can use the `rmag.process_hyst_loops` function to process all the loops.

In [6]:
results = rmag.process_hyst_loops(hyst_experiments, measurements)

## export hysteresis loop statistics into a MagIC specimens data table

In [7]:
rmag.add_hyst_stats_to_specimens_table(specimens, 'IRM-VSM3-LP-HYS-218845', NED1_5c_hyst_process_result)